In [1]:
import pandas as pd
import numpy as np
import re
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam




In [2]:
# Load the dataset
df = pd.read_csv(r'C:\Users\ATHULRAJ B C\Downloads\train.csv')  # Assuming the dataset file name is 'twitter_data.csv'
df.head()


,Tweets,label
0,Asked #ChatGPT about what it thinks are the pr...,Positive
1,#ChatGPT tornado has already traveled around t...,Neutral
2,This is a great explanation of why #EVs are mo...,Positive
3,‘if you need to write a box-ticking social med...,Positive
4,Just saw an AI tool making my coffee for me. \...,Positive


In [3]:
print(df["Tweets"].dtype)


object


In [4]:
df["Tweets"] = df["Tweets"].astype(str)


In [5]:
def clean_text(text):
    text = re.sub(r"[^a-zA-Z0-9\s]", "", text)
    text = text.lower()
    text = re.sub(r"\s+", " ", text)
    return text

df["Cleaned_Tweets"] = df["Tweets"].apply(clean_text)


In [6]:
df.head()

,Tweets,label,Cleaned_Tweets
0,Asked #ChatGPT about what it thinks are the pr...,Positive,asked chatgpt about what it thinks are the pro...
1,#ChatGPT tornado has already traveled around t...,Neutral,chatgpt tornado has already traveled around th...
2,This is a great explanation of why #EVs are mo...,Positive,this is a great explanation of why evs are mor...
3,‘if you need to write a box-ticking social med...,Positive,if you need to write a boxticking social media...
4,Just saw an AI tool making my coffee for me. \...,Positive,just saw an ai tool making my coffee for me ch...


In [9]:
print(df.columns)


Index(['Tweets', 'label', 'Cleaned_Tweets'], dtype='object')


In [10]:
df["sentiment"] = pd.Series([None] * len(df))  # Fill with None or placeholder


In [11]:
sentiment = {"positive": 1, "neutral": 0, "negative": -1}
# Ensure your column values are exactly "positive", "neutral", or "negative"


In [12]:
df["label"] = df["label"].map(sentiment)

In [18]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df["Cleaned_Tweets"], df["label"], test_size=0.2)


In [27]:
df.head()

,Tweets,label,Cleaned_Tweets
0,Asked #ChatGPT about what it thinks are the pr...,Positive,asked chatgpt about what it thinks are the pro...
1,#ChatGPT tornado has already traveled around t...,Neutral,chatgpt tornado has already traveled around th...
2,This is a great explanation of why #EVs are mo...,Positive,this is a great explanation of why evs are mor...
3,‘if you need to write a box-ticking social med...,Positive,if you need to write a boxticking social media...
4,Just saw an AI tool making my coffee for me. \...,Positive,just saw an ai tool making my coffee for me ch...


In [20]:
tokenizer = Tokenizer(num_words=10000)  
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

maxlen = max(len(seq) for seq in X_train_seq)  # Maximum sequence length
X_train_pad = pad_sequences(X_train_seq, maxlen=maxlen)
X_test_pad = pad_sequences(X_test_seq, maxlen=maxlen)


In [21]:
model = Sequential()
model.add(Embedding(10000, 128, input_length=maxlen))  # Adjust embedding dimensions
model.add(LSTM(64, return_sequences=True))  # Adjust LSTM units
model.add(LSTM(32))  # Adjust LSTM units
model.add(Dense(3, activation="softmax"))  # Output layer for 3 sentiment classes

model.compile(loss="sparse_categorical_crossentropy", optimizer=Adam(learning_rate=0.001))
